In [1]:
! pip install dash-dangerously-set-inner-html

In [2]:
# Global import

import datetime as dt

import pandas as pd
import numpy as np
import plotly.graph_objects as go

from dash_mdc_neptune import themes

In [3]:
# Global variables

ASSET_NAME = "PWRTE"
PORTFOLIO_NAME = "TEST_FR"

# PAGE 1

In [4]:
# Retrieve trades

query_params = {
    "asset": ASSET_NAME,
    "portfolio_name": PORTFOLIO_NAME,
}

trades_df = nptc.filter_forward_contracts(**query_params)

In [5]:
# Format trades

all_df = trades_df[npta.definitions.TRADE_DATE_COLUMNS]
latest_df = all_df[all_df.traded_at==all_df.traded_at.max()]

## SECTION 1

### CARD 1

In [6]:
all_styler = all_df.style
all_styler = all_styler.set_table_styles(
    [
        themes.pandas.noos_pd_row_hover,
        themes.pandas.noos_pd_headers,
    ]
)
all_styler = all_styler.hide_index()

In [7]:
latest_styler = latest_df.style
latest_styler = latest_styler.set_table_styles(
    [
        themes.pandas.noos_pd_row_hover,
        themes.pandas.noos_pd_headers,
    ]
)
latest_styler = latest_styler.hide_index()

### CARD 2

In [8]:
trades_volume_df = npta.get_volumes_and_costs_from_trades(all_df)
latest_volumes_df = npta.get_volumes_and_costs_from_trades(latest_df)

In [9]:
# Shared heatmap configs

GO_LAYOUT = go.Layout(
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text='local delivery date',
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='MWh',
        ),
        side="left",
        overlaying='y2',
    ),
    yaxis2=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text='EUR/MWh',
        ),
        side="right",
    ),
    template="noos_watermark+noos_base+hoover_xunified+xaxis_period+min_modebar",
)

In [10]:
all1 = go.Scatter(
    x = trades_volume_df.index,
    y = trades_volume_df.total_volume * 2,
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = True
)

all2 = go.Scatter(
    x = trades_volume_df.index,
    y = trades_volume_df.price_average,
    name = "Prices",
    visible = True,
    yaxis="y2"
)

latest1 = go.Scatter(
    x = latest_volumes_df.index,
    y = latest_volumes_df.total_volume * 2,
    name = "Volumes",
    fill='tozeroy',
    yaxis="y",
    visible = False
)

latest2 = go.Scatter(
    x = latest_volumes_df.index,
    y = latest_volumes_df.price_average,
    name = "Prices",
    visible = False,
    yaxis="y2"
)

fig = go.FigureWidget(
    data=[all1, all2, latest1, latest2],
    layout=GO_LAYOUT,
)

## Definitions:

 - **Volumes** represent aggregated volumes per delivery period.
 - **Prices** represent average traded prices per delivery period

# DASH

In [11]:
import jupyter_dash

import dash_dangerously_set_inner_html as html
from dash import dcc

import dash_mdc_neptune as mdc

In [24]:
radio = dcc.RadioItems(
    id="radio",
    options=[
        {'label': 'All trades', 'value': 'ALL'},
        {'label': 'Latest only', 'value': 'LATEST'},
    ],
    value='ALL'
)

In [33]:
table = html.DangerouslySetInnerHTML(children=all_styler.to_html())

extras = {
    "style": {"height": "100%", "width": "100%"},
}
graph = dcc.Graph(
    id="graph",
    figure=fig,
    **extras
)

tab = mdc.Tab(
    id="tab",
    children=[table, graph],
    options=[
        {"label": "Trade blotter"},
        {"label": "Trade profile"},
    ],
)
section = mdc.Section(
    id="section",
    orientation="columns",
    children=[radio, tab],
    options=[
        {"title": "", "size": 2},
        {"title": ""},
    ],
)

page = mdc.Page(children=section)
navbar = mdc.NavBar(title="Wholesale Hedge Trades")

layout = mdc.Dashboard(children=[navbar, page])

In [34]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output


app = JupyterDash(__name__)
app.layout = layout


@app.callback(
    Output(component_id='tab', component_property='children'),
    Input(component_id='radio', component_property='value')
)
def update_tabs(value):
    if value == 'ALL':
        table = html.DangerouslySetInnerHTML(all_styler.to_html())
        fig.data[2].visible=False
        fig.data[3].visible=False
        fig.data[0].visible=True
        fig.data[1].visible=True
    else:
        fig.data[0].visible=False
        fig.data[1].visible=False
        fig.data[2].visible=True
        fig.data[3].visible=True
        table = html.DangerouslySetInnerHTML(latest_styler.to_html())
    extras = {
        "style": {"height": "100%", "width": "100%"},
    }
    graph = dcc.Graph(
        id="graph",
        figure=fig,
        **extras
    )
    return [table, graph]

app.run_server(mode='external', host="0.0.0.0", port=8001)

Dash app running on http://0.0.0.0:8001/


/opt/conda/lib/python3.8/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

